In [28]:
import pickle
path = "data/Physics/0_uncopynode.pkl"
with open(path, 'rb') as f:
    data = pickle.load(f)
print(data.edge_index.max())
print(len(data.x))

tensor(4440)
4441


In [41]:
# print(data.edge_index.shape, data.train_mask.shape)
# s = set()
# for e in data.edge_index[1]:
#     s.add(e.item())
# print(len(s))
# data.edge_index
# data.train_mask

# path = 'data/SBM/0_uncopynode.pkl'
# with open(path, 'rb') as f:
#     data = pickle.load(f)
# print(data.y.min(),data.y.max())
# print(data.edge_index[0].max())
# print(data.edge_index[1].max())
# print(len(data.x[0]))
import torch
x = torch.Tensor([0,0])
print(x.to('cuda').grad)


None


In [6]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
with open(path_sbm_train, 'rb') as f:
    sbm_train = pickle.load(f)
with open(path_sbm_val, 'rb') as f:
    sbm_val = pickle.load(f)
with open(path_sbm_test, 'rb') as f:
    sbm_test = pickle.load(f)
# sbm_train

In [8]:
# for tmp in sbm_train:
#     print(len(tmp.node_feat))
# sbm_train[0].node_feat
# edge_index = torch.tensor([[],[]])
# print(edge_index)
# print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# x = torch.BoolTensor([0]*10)
# x[1:3] = True
# print(x)
# print([1,2]*2)
s_train = 0
for tmp in sbm_train:
    s_train += len(tmp.node_feat)
s_val = 0
for tmp in sbm_val:
    s_val += len(tmp.node_feat)
s_test = 0
for tmp in sbm_test:
    s_test += len(tmp.node_feat)
print(s_train, s_val, s_test, s_train + s_val+s_test)

1168087 116896 116820 1401803


In [18]:
from torch_geometric.data import Data
K=100
NBTRAIN = len(sbm_train)
NBVAL = len(sbm_val)
NBTEST = len(sbm_test)

import torch
for k in range(K):
    print(k)
    idx_base=0
    edge_index = torch.LongTensor([[],[]])
    x = torch.tensor([])
    y = torch.LongTensor([])
    for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
        sub_graph = sbm_train[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_train = idx_base
    for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
        sub_graph = sbm_val[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_val = idx_base
    for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
        sub_graph = sbm_test[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sbm_train[0].W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    idx_test = idx_base

    x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
    train_mask = torch.BoolTensor([0]*idx_base)
    train_mask[0:idx_train] = True
    val_mask = torch.BoolTensor([0]*idx_base)
    val_mask[idx_train:idx_val] = True
    test_mask = torch.BoolTensor([0]*idx_base)
    test_mask[idx_val:idx_test] = True
    with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
        pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = val_mask), f)


0
1
2
3


/tmp/ipykernel_44285/1685744141.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [30]:
from models import Gat
model = Gat(6,5)
path = 'data/SBM/0_uncopynode.pkl'
with open(path, 'rb') as f:
    data = pickle.load(f)
output = model(data.x,data.edge_index)
output

tensor([[-2.5087e-03, -1.4158e-05, -1.2516e-02,  2.1653e-03,  1.3335e-03],
        [-2.4372e-03,  5.1154e-04, -1.1835e-02,  6.4980e-04,  4.5894e-04],
        [-2.8227e-03, -1.7505e-03, -1.3460e-02,  2.1013e-03,  9.9353e-04],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       grad_fn=<AsStridedBackward>)